In [28]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn import datasets, linear_model
from sklearn.cross_validation import KFold    
import sklearn
from sklearn.metrics import mean_squared_error
from numpy.linalg import inv
from sklearn.preprocessing import PolynomialFeatures


# THIS IS TO READ FROM A DAT FILE AND GET A LIST OF LIST WITH WITH EACH COULOUMN AS A LIST
getListFromAFile= lambda filename:np.loadtxt(filename, unpack=True)

# returning the list of all the file  in the folder
getListOfFiles = lambda directoryPath :[f for f in os.listdir(directoryPath) if os.path.isfile(os.path.join(directoryPath, f))] # this is to list all files

# GENRAL CUSTOM 2D PLOTS SCATTER PLOTS
def createPlots(inputList_x,inputList_y,xlabel="x-axis -->",ylabel="y-axis -->",title="file-Name-Title",plotterRef=None):
    #this is for linear regression
    if plotterRef is not None:
        plt.scatter(inputList_x,plotterRef(inputList_x),alpha=0.5,color ='r')   
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.scatter(inputList_x,inputList_y,alpha=0.5,color ='b')
    plt.show()

def showDataInFile(directoryPath):
    with open("Data1/"+directoryPath,'r') as f:
        next(f) # skip first row
        df = pd.DataFrame(l.rstrip().split() for l in f)
        df.boxplot
        print(df)

# this like the the execution main
# displaying all the files in Data/ folder  and ploting them
for s in getListOfFiles('Data1/'):
    data = getListFromAFile("Data1/"+s)
    xdata = np.copy(data[:-1,:])
    ydata = np.copy(data[-1,:])
    print "you are printing the xdata for file "+s
    #print xdata.T
    print "you are printing the ydata"
    print ydata.T
   

you are printing the xdata for file mvar-set1.dat
you are printing the ydata
[ 2.86198097  0.51952523 -2.24576096 ..., -0.66468951 -0.10755482
  2.20162969]
you are printing the xdata for file mvar-set2.dat
you are printing the ydata
[-0.10825881 -0.01290997 -0.12720998 ..., -0.05667539 -0.14304067
  0.16316383]
you are printing the xdata for file mvar-set3.dat
you are printing the ydata
[ 1.50730521 -4.40413503 -1.52724025 ...,  5.47160025  2.92036708
  2.32869529]
you are printing the xdata for file mvar-set4.dat
you are printing the ydata
[ 0.23370968  0.02344994 -0.03993024 ...,  0.04048006 -0.03316965
  0.00083013]


In [26]:
class PolyExplicit:
    #A x = B    
    def createtor(self):
        poly = PolynomialFeatures(self.degree)
        Z = poly.fit_transform(self.xlist)
        self.coeff=np.dot(inv(np.dot(Z.T,Z)),np.dot(Z.T,self.ylist))
    
    def fit(self,degree,xlist,ylist):
        
        assert(degree>0)
        self.degree = degree
        self.xlist  = xlist
        self.ylist  = ylist
        self.createtor()
    
    def predict(self,getList):
        inList = getList
        #print inList
        return [sum ([self.coeff[j+1]*lister[j]for j in range(0,len(lister))])+self.coeff[0] for lister in inList]
        
errors =[]
kFolds =10
for s in getListOfFiles('Data1/'):
    data = getListFromAFile("Data1/"+s)
    xdata = np.copy(data[:-1,:])
    ydata = np.copy(data[-1,:])
    kf = sklearn.cross_validation.KFold(n=len(ydata), n_folds=kFolds, shuffle=False,random_state=None)
    error = 0
    regression = PolyExplicit()
    for train_index, test_index in kf:
        X_train, X_test =xdata.T[train_index], xdata.T[test_index]
        y_train, y_test = ydata[train_index], ydata[test_index]
        regression.fit(1,X_train,y_train)
        error = error + mean_squared_error(y_test,regression.predict(X_test))
    error = error/kFolds
    errors.append({'PolyNomial degree':1,'file':s,'mse':error})

df = pd.DataFrame(errors)
print df

   PolyNomial degree           file       mse
0                  1  mvar-set1.dat  0.259149
1                  1  mvar-set2.dat  0.019957
2                  1  mvar-set3.dat  0.250781
3                  1  mvar-set4.dat  0.004190


In [34]:
class PolyIterativeGradientDecent:
    #A x = B    
    def createtor(self):
        poly = PolynomialFeatures(self.degree)
        Z = poly.fit_transform(self.xlist)
        self.coeff=self.gradient_descent_2(self.alpha, Z, self.ylist, self.numIterations)
        print self.coeff
    
    def fit(self,degree,xlist,ylist,alpha=0.0001,numIterations=4000):
        assert(degree>0)
        self.degree = degree
        self.xlist  = xlist
        self.ylist  = ylist
        self.alpha = alpha
        self.numIterations = numIterations
        self.createtor()
    
    def predict(self,getList):
        inList = getList
        return [sum ([self.coeff[j+1]*lister[j]for j in range(0,len(lister))])+self.coeff[0] for lister in inList]
    def gradient_descent_2(self,alpha, x, y, numIterations):
        m = len(xdata[0]) # number of samples
        theta = np.ones(len(xdata)+1)
        x_transpose = x.T
        for iter in range(0, numIterations):
            hypothesis = np.dot(x, theta)
            loss = hypothesis - y
            J = np.sum(loss ** 2) / (2 * m)  # cost
            #print "iter %s | J: %.3f" % (iter, J)      
            gradient = np.dot(x_transpose, loss) / m         
            theta = theta - alpha * gradient  # update
        return theta
        
errors =[]
kFolds =10
for s in getListOfFiles('Data1/'):
    data = getListFromAFile("Data1/"+s)
    xdata = np.copy(data[:-1,:])
    ydata = np.copy(data[-1,:])
    kf = sklearn.cross_validation.KFold(n=len(ydata), n_folds=kFolds, shuffle=False,random_state=None)
    error = 0
    regression = PolyIterativeGradientDecent()
    for train_index, test_index in kf:
        X_train, X_test =xdata.T[train_index], xdata.T[test_index]
        y_train, y_test = ydata[train_index], ydata[test_index]
        regression.fit(1,X_train,y_train)
        error = error + mean_squared_error(y_test,regression.predict(X_test))
    error = error/kFolds
    errors.append({'Gradient decent with features':len(xdata),'file':s,'mse':error})


df = pd.DataFrame(errors)
print df

[ 1.00009847  0.99948903  0.99785714]
[ 0.99840539  0.99907592  0.99796578]
[ 0.99847951  0.99792459  0.9960769 ]
[ 0.99784116  0.99863889  0.99628236]
[ 0.99849532  1.00048944  0.99614515]
[ 0.99975657  0.99798117  0.99631957]
[ 0.99861203  0.99842033  0.99625319]
[ 0.99986642  0.99956399  0.99649114]
[ 0.99765986  0.99961561  0.99540088]
[ 0.99832495  0.99906731  0.99378823]
[ 0.69658657  0.62734929  0.60607917]
[ 0.6979882   0.62743716  0.59857425]
[ 0.69579827  0.63235445  0.60846657]
[ 0.70199519  0.63636787  0.60979004]
[ 0.70073821  0.63668238  0.60761555]
[ 0.69221817  0.62752077  0.60464321]
[ 0.69897009  0.63462673  0.60820529]
[ 0.69603988  0.63218772  0.60299789]
[ 0.69819382  0.63494659  0.6091599 ]
[ 0.70088533  0.63218597  0.60975557]
[ 0.99979891  0.99997924  0.99974642  0.99982875  0.54716502  1.45083973]
[ 1.00064293  0.999994    1.00067585  0.99981092  0.54794589  1.45119383]
[ 0.99975014  0.99931253  1.00126195  1.00012273  0.54695595  1.45194105]
[ 0.99940944  0.99